In [ ]:
pivot = WIKIP2000.History.pivot(index='Date',columns='Symbol',values='Close').isnull()
pivot = pivot.reset_index()
unpivot = pd.melt(pivot, id_vars = 'Date', value_vars=pivot.columns[1:])

missing_data = unpivot[unpivot.value == True]

subdatas = []

# For each identified missing date/symbol pair in missing_data, identify the most recent previous existing record and
# substitute that value in its place.
for i in range(len(missing_data)):
    breaker = 0
    symbol = missing_data.iloc[i].Symbol
    date = missing_data.iloc[i].Date.to_pydatetime()

    if date.weekday() == 0:
        back = 3
    elif date.weekday() == 6:
        back = 2
    else:
        back = 1

    prevdate = date + datetime.timedelta(days=-back)

    while WIKIP2000.History[(WIKIP2000.History.Date == prevdate) & (WIKIP2000.History.Symbol == symbol)].shape[0] == 0:

        prevdate = prevdate + datetime.timedelta(days=-1)

        if prevdate <= min(WIKIP2000.History.Date):

            WIKIP2000.History = WIKIP2000.History[~(WIKIP2000.History.Symbol == symbol)]
            breaker = 1
            break

    if breaker == 1:
        print('Data Substitution Failed. All {} data removed from History'.format(symbol))
        pass
    else:
        ind = WIKIP2000.History[(WIKIP2000.History.Date == prevdate) & (WIKIP2000.History.Symbol == symbol)].index.values[0]
        subdata = WIKIP2000.History[WIKIP2000.History.index == (ind)].to_dict('records')[0]
        subdata['Date'] = date
        subdata['DailyPctChg'] = 0
        subdatas.append(subdata)

# Lastly, append all the new (substitution) records onto the old History DataFrame, and then reorganize so we're back in order.
WIKIP2000.History = pd.concat([WIKIP2000.History, pd.DataFrame(subdatas)], ignore_index = True)
WIKIP2000.History = WIKIP2000.History.sort_values(by=['Symbol','Date'])
print('{} at {} substituted with data from {}'.format(symbol,date,prevdate))